In [2]:
import tensorflow as tf

In [40]:
import numpy as np

In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


Layers: common sets of useful operations
Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.


In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
layer.input_spec

InputSpec(min_ndim=2)

In [9]:
layer.dtype

'float32'

In [8]:
layer.units

10

In [6]:
layer.name

'dense_1'

It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [12]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [13]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<KerasVariable shape=(5, 10), dtype=float32, path=dense_1/kernel>,
 <KerasVariable shape=(10,), dtype=float32, path=dense_1/bias>]

In [14]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<KerasVariable shape=(5, 10), dtype=float32, path=dense_1/kernel>,
 <KerasVariable shape=(10,), dtype=float32, path=dense_1/bias>)

Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

__init__ , where you can do all input-independent initialization
build, where you know the shapes of the input tensors and can do the rest of the initialization
call, where you do the forward computation
Note that you don't have to wait until build is called to create your variables, you can also create them in __init__. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.


class MyDenseLayer(tf.keras.laye

In [41]:
# Define the layer
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name='kernel',
            shape=(int(input_shape[-1]), self.num_outputs),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            name='bias',
            shape=(self.num_outputs,),
            initializer='zeros',
            trainable=True
        )

    # def call(self, inputs):
    #     return tf.matmul(inputs, self.kernel) + self.bias
    
    def call(self, inputs):
        return np.dot(inputs, self.kernel) + self.bias

In [42]:
# Create object
layer = MyDenseLayer(num_outputs=2)

In [63]:
layer3 = MyDenseLayer(num_outputs=2)

In [ ]:

# Build with input shape
input_shape = (2,4,)  # Example input shape tensor  # input shape required or no of features columns no =4 row 2
layer3.build(input_shape)

In [ ]:
# input_shape = (4,)  # input shape required or no. of features columns no. =4 for tensor

In [65]:
input_shape[-1]

4

In [ ]:
# Verify weights
print(layer3.kernel.shape) # columns x no. of output or class # Output: (784, 64)
print(layer3.bias.shape)    # Output: (64,)


#   1x4 * 4x2 + 1x4 = 1x2 + 1x2
# [ 2 3 4 5 ] * [[6 7],    + [ 8 8 ]
#        1x4     [8 9],
#                [10 11],
#                [12 13]]
#                     4x2

(4, 2)
(2,)


In [67]:
layer3.kernel

<KerasVariable shape=(4, 2), dtype=float32, path=my_dense_layer_5/kernel>

In [68]:
layer3.num_outputs

2

In [69]:
input_shape[-1]

4

In [70]:
layer3.trainable_variables

[<KerasVariable shape=(4, 2), dtype=float32, path=my_dense_layer_5/kernel>,
 <KerasVariable shape=(2,), dtype=float32, path=my_dense_layer_5/bias>]

In [71]:
inputs = np.array([[1, 2, 3, 6],[1, 2, 3,6]])

In [72]:
layer3.call(inputs)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2928\2178694524.py:25: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  return np.dot(inputs, self.kernel) + self.bias


array([[-1.10742164, -6.01397204],
       [-1.10742164, -6.01397204]])

In [53]:
inputstensor = tf.convert_to_tensor(inputs)

In [54]:
inputstensor

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 2, 3, 6])>

In [ ]:
layer.call(inputstensor)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2928\2178694524.py:25: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  return np.dot(inputs, self.kernel) + self.bias


array([-7.00791311, -5.10822439])

In [51]:
inputs

array([1, 2, 3, 6])

In [52]:
layer.call(inputs)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2928\2178694524.py:25: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  return np.dot(inputs, self.kernel) + self.bias


array([-7.00791311, -5.10822439])

In [20]:
tf.zeros([10, 5])

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [56]:
print([var.name for var in layer.trainable_variables])

['kernel', 'bias']


In [57]:
layer(np.array([1, 2, 3, 6]))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2928\2178694524.py:25: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  return np.dot(inputs, self.kernel) + self.bias


array([-7.00791311, -5.10822439])

Models: Composing layers
Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from keras.Model when you need the model methods like: Model.fit,Model.evaluate, and Model.save (see Custom Keras layers and models for details).

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1)) # no of  filters, shape of filter
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)




ow Batch Normalization Works
Calculate Mean and Variance: Compute mean (μ) and variance (σ^2) of activations for each mini-batch.
Normalize Activations: Subtract mean, divide by sqrt(variance + epsilon).
Scale and Shift: Multiply by gamma (γ), add beta (β).
Batch Normalization Formula
Normalized activation = γ * ((x - μ) / sqrt(σ^2 + ε)) + β

In [ ]:
block = ResnetIdentityBlock(1, [1, 2, 3])

In [51]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [52]:
block.layers

[<Conv2D name=conv2d, built=True>,
 <BatchNormalization name=batch_normalization, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <BatchNormalization name=batch_normalization_1, built=True>,
 <Conv2D name=conv2d_2, built=True>,
 <BatchNormalization name=batch_normalization_2, built=True>]

In [53]:
len(block.variables)

18

In [54]:
block.summary()

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (1, 2, 3, 1)           │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (1, 2, 3, 1)           │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (1, 2, 3, 2)           │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (1, 2, 3, 2)           │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (1, 2, 3, 3)           │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (1, 2, 3, 3)           │            12 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)

In [55]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [56]:
my_seq.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, None, None, 1)  │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, None, None, 1)  │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, None, None, 2)  │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, None, None, 2)  │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, None, None, 3)  │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, None, None, 3)  │            12 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)